In [19]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline  

In [22]:
n_input_width = 28
n_input_height = 28
n_input_channel = 1
training_epochs = 100
batch_size = 500
learning_rate = 0.01

n_conv1_patch_size = 3 
n_conv1_filter = 84

n_output  = 10 # e.g. MNIST total classes (0-9 digits)

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input_width * n_input_height])
y = tf.placeholder(tf.float32, [None, n_output])

wc1 = tf.Variable(tf.random_normal([n_conv1_patch_size, n_conv1_patch_size, n_input_channel, n_conv1_filter], stddev=0.1))
bc1 = tf.Variable(tf.random_normal([n_conv1_filter], stddev=0.1))

wf1 = tf.Variable(tf.random_normal([(n_input_width/2)*(n_input_height/2)*n_conv1_filter, n_output], stddev=0.1))
bf1 = tf.Variable(tf.random_normal([n_output], stddev=0.1))

# Reshape input
input_r = tf.reshape(x, shape=[-1, n_input_width, n_input_width, 1])
# Convolution
conv = tf.nn.conv2d(input_r, wc1, strides=[1, 1, 1, 1], padding='SAME')
# Add-bias
bias = tf.nn.bias_add(conv, bc1)
# Pass ReLu
relu = tf.nn.relu(bias)
# Max-pooling
pool  = tf.nn.max_pool(relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
# Vectorize
dense = tf.reshape(pool, [-1, wf1.get_shape().as_list()[0]])
# Fully-connected layer
out = tf.add(tf.matmul(dense, wf1), bf1)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz



In [ ]:
-

Epoch   0 , Minibatch Loss= 6.036530, Training Accuracy= 0.15600
Epoch   1 , Minibatch Loss= 11.040858, Training Accuracy= 0.19200
Epoch   2 , Minibatch Loss= 11.840248, Training Accuracy= 0.09400
Epoch   3 , Minibatch Loss= 6.688283, Training Accuracy= 0.28200
Epoch   4 , Minibatch Loss= 5.790600, Training Accuracy= 0.11800
Epoch   5 , Minibatch Loss= 4.669760, Training Accuracy= 0.13200
Epoch   6 , Minibatch Loss= 3.899807, Training Accuracy= 0.20400
Epoch   7 , Minibatch Loss= 3.039432, Training Accuracy= 0.24800
Epoch   8 , Minibatch Loss= 2.742738, Training Accuracy= 0.27200
Epoch   9 , Minibatch Loss= 2.412395, Training Accuracy= 0.24800
Epoch   10 , Minibatch Loss= 2.118133, Training Accuracy= 0.34800
Epoch   11 , Minibatch Loss= 1.741504, Training Accuracy= 0.35400
Epoch   12 , Minibatch Loss= 1.324258, Training Accuracy= 0.49400
Epoch   13 , Minibatch Loss= 1.231168, Training Accuracy= 0.56800
Epoch   14 , Minibatch Loss= 0.898669, Training Accuracy= 0.71800
Epoch   15 , Minib